#### 라이브러리

In [1]:
import os
import re
import torch
import requests
import pandas as pd
import numpy as np

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import RetrievalQA

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

#### csv 파일 로드

In [ ]:
# cleaned_word_dict.csv 파일 확인
df = pd.read_csv("./cleaned_word_dict.csv")
df.head()

,Word,Content
0,가계부실위험지수(HDRI),가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 부실...
1,가계수지,가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 표시한...
2,가계순저축률,일반적으로 저축률은 저축액을 처분가능소득으로 나눈 비율을 말한다. 마찬가지로 가계순...
3,가계신용통계,가계신용통계는 가계부문에 대한 신용공급 규모를 나타내는 통계이다. 가계신용은 금융기...
4,가계처분가능소득,가계처분가능소득(PDI; Personal Disposable Income)은 가계가...


In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="./cleaned_word_dict.csv")
docs = loader.load()
print(f"문서의 수: {len(docs)}")

print(f"\n[페이지내용]\n{docs[10].page_content[:500]}")
print(f"\n[metadata]\n{docs[10].metadata}\n")

문서의 수: 700

[페이지내용]
﻿Word: 가상통화공개(ICO)
Content: 가상통화(ICO; Initial Coin Offering) 공개는 주로 혁신적인 신생기업(startup)이 암호 화화폐(cryptocurrency) 또는 디지털 토큰(digital token, 일종의 투자증명)을 이용하여 자금을 조달할 수 있는 크라우드펀딩(crowd funding)의 한 방식이다. 가상통화공개 (ICO)에서 새로 발행된 암호화화폐는 법화(legal tender) 또는 비트코인 등 기존의 가상 통화와 교환되어 투자자에게 팔린다. 이 용어는 거래소에 상장하려는 기업이 투자자에게 자기 주식을 처음 공개적으로 매도하는 기업공개(IPO; Initial Public Offering)에서 연유 되었다고 볼 수 있다. 기업공개(IPO)에 참여한 투자자는 해당 기업의 소유권과 관련하여 주식을 획득한다. 반면 가상통화공개(ICO)에 참여한 투자자는 해당 신생기업의 코인 (coins) 또는 토큰을 얻는데, 이는 해당 기업이 제안한 프로

[metadata]
{'source': './cleaned_word_dict.csv', 'row': 10}



#### 데이터 분할

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"청크 수: {len(split_documents)}")

청크 수: 721


#### 임베딩 & 벡터스토어 생성

In [6]:
# generate Chroma VectorStore
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = Chroma.from_documents(
    documents=split_documents, 
    embedding=embedding_model,
    persist_directory="chroma_index"
)

vectorstore.persist()

C:\Users\lgpc\AppData\Local\Temp\ipykernel_20716\147296882.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
C:\Users\lgpc\AppData\Local\Temp\ipykernel_20716\147296882.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


#### 추가 pdf 파싱 후 Chroma에 추가

In [9]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./[2024 한권으로 OK 주식과 세금].pdf")
docs = loader.load()

In [10]:
output_file = "./[2024 한권으로 OK 주식과 세금].md"
with open(output_file, "w", encoding="utf-8") as f:
    for i, doc in enumerate(docs, start=1):
        f.write(f"## Page {i}\n")  
        f.write(doc.page_content) 
        f.write("\n\n")     

In [11]:
# text split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"청크 수: {len(split_documents)}")

청크 수: 220


In [12]:
# load Chroma VectorStore
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = Chroma(
    persist_directory="chroma_index", embedding_function=embedding_model
)

C:\Users\lgpc\AppData\Local\Temp\ipykernel_20716\3454874282.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [13]:
# add split_documents to Chroma
vectorstore.add_documents(split_documents)

# save
vectorstore.persist()

#### Retriever 생성 & LLM 모델 불러오기

In [7]:
# load Chroma VectorStore
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = Chroma(
    persist_directory="chroma_index", embedding_function=embedding_model
)

C:\Users\lgpc\AppData\Local\Temp\ipykernel_15860\2903721051.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
C:\Users\lgpc\AppData\Local\Temp\ipykernel_15860\2903721051.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [8]:
# define retriever
retriever = vectorstore.as_retriever(search_kwargs={"k":5})

# load Gemini
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [9]:
# Function to fetch news from Naver API
def fetch_naver_news(query):
    client_id = os.getenv("NAVER_CLIENT_ID")
    client_secret = os.getenv("NAVER_CLIENT_SECRET")
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display=3&sort=sim"
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret,
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        news_items = response.json().get("items", [])
        if not news_items:
            return "관련 뉴스를 찾을 수 없습니다. 보다 구체적인 검색어를 사용해보세요."
        return "\n".join([f"- {item['title']} ({item['link']})" for item in news_items])
    return "네이버 뉴스 API 요청 실패. 응답 코드: " + str(response.status_code)

In [27]:
template = """
당신은 금융 전문가입니다.
사용자가 입력한 금융 용어를 누구나 이해하기 쉽고 간단하게 설명하고, 3개의 연관 검색어를 제공합니다.
그리고 금융 용어 이해를 돕기 위한 관련 뉴스와 링크를 제공합니다.

관련 정보:
{context}

금융 용어:
{term}

설명: 

연관 검색어:

관련 뉴스: {news}
"""

In [28]:
# generate prompt
prompt = PromptTemplate(
    input_variables=["context", "news", "term"], 
    template=template
)

# generate Chain
chain = (
    {"context":retriever, "term":RunnablePassthrough(), "news":fetch_naver_news}
    | prompt
    | llm 
    | StrOutputParser()
)

In [29]:
term = "ETF"
response = chain.invoke(term)
print(response)

**ETF (상장지수펀드)**란 무엇일까요?

쉽게 말해, 특정 지수(예: 코스피200, S&P500)의 움직임을 따라가도록 설계된 펀드를 주식처럼 거래소에서 사고 팔 수 있도록 만든 상품입니다.  주식처럼 거래되기 때문에 언제든지 매수/매도가 가능하고,  다양한 지수를 추종하는 ETF가 있어서 원하는 시장에 손쉽게 투자할 수 있다는 장점이 있습니다.  개별 주식을 일일이 고르는 수고를 덜고, 분산투자를 통해 위험을 줄일 수 있다는 점도 큰 매력입니다.


**연관 검색어:**

1. 인덱스펀드
2. 레버리지 ETF
3. ETN (상장지수증권)


**관련 뉴스:**

* **ETF 업계 '지각변동'…연초부터 점유율·인재영입 경쟁 가열 (https://n.news.naver.com/mnews/article/001/0015183501?sid=101):**  ETF 시장의 경쟁이 치열해지고 있음을 보여주는 뉴스입니다.  다양한 ETF 상품 출시와 운용사 간의 경쟁이 심화되고 있다는 것을 알 수 있습니다.

* **금리인하 시기 주목할 ETF는..장기채·리츠 주목 (https://n.news.naver.com/mnews/article/014/0005301497?sid=101):**  시장 상황에 따라 어떤 ETF에 투자하는 것이 유리한지에 대한 정보를 제공하는 뉴스입니다. 금리 인하 시대에 적합한 ETF 투자 전략을 제시하고 있습니다.

* **‘TR ETF’ 7월부터 과세… 어떻게 바뀌나 (https://n.news.naver.com/mnews/article/022/0004006484?sid=101):**  ETF 투자와 관련된 세금 정책 변화에 대한 뉴스입니다.  특정 ETF에 대한 과세 방식이 변경됨에 따라 투자 전략을 수정해야 할 필요성을 강조하고 있습니다.


In [30]:
term = "가계수지"
response = chain.invoke(term)
print(response)

## 가계수지: 우리 집의 돈 관리 보고서

**설명:** 가계수지는 가정에서 일정 기간(보통 1달) 동안 벌어들인 돈(소득)과 쓴 돈(지출)을 모두 합쳐서 비교해 본 것입니다.  소득이 지출보다 많으면 '흑자', 지출이 소득보다 많으면 '적자'라고 합니다.  흑자는 저축을 했다는 뜻이고, 적자는 빚을 졌거나 저축을 까먹었다는 뜻이죠.  마치 개인 회사의 재무제표처럼, 가계의 경제 상황을 한 눈에 보여주는 중요한 지표입니다.

**연관 검색어:**  가계부, 소비지출, 저축률

**관련 뉴스:**

* **朴탄핵-尹계엄 경제지표 비교한 KDI…&quot;9년 전보다 가계·기업심리 위축... (https://www.etoday.co.kr/news/view/2435818)**:  정치적 상황 변화가 가계수지에 어떤 영향을 미치는지 보여주는 뉴스입니다.  정치적 불안정이 가계의 소비심리와 투자심리를 위축시켜 가계수지에 부정적 영향을 줄 수 있음을 시사합니다.
* **KDI &quot;국내 정황으로 가계·기업 심리지표 악화…경기 하방 위험 증... (https://www.newspim.com/news/view/20250108000236)**:  국내 정치·경제 상황이 가계 심리와 경제 활동에 미치는 영향을 분석한 뉴스입니다. 가계 심리 악화는 가계수지의 적자 확대 가능성을 높입니다.
* **경상수지 흑자 ‘반갑다’…올해 900억 고지 달성하나[한은 미리보기] (https://n.news.naver.com/mnews/article/018/0005918443?sid=101)**:  경상수지는 국가 전체의 수입과 지출을 나타내는 지표입니다.  가계수지와는 규모와 범위가 다르지만, 국가 경제 상황이 가계 경제에 영향을 미치므로 관련 뉴스를 참고하는 것이 도움이 됩니다.  경상수지 흑자는 국가 경제가 안정적이라는 것을 의미하며, 이는 긍정적으로 가계수지에도 영향을 줄 수 있습니다.


**(추가)**  제공된 뉴스 링크 중 하나는 경상수지 관련 뉴스입니다.  가계수지는 개별 